In [6]:
#import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import os
from keras_facenet import FaceNet

import sys


from IPython.utils import io

import cv2

#print the number of GPUs available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:

pathLFW = './LFW/lfw_funneled/'

#take the first 100 folders
folders = os.listdir(pathLFW)[:100]

#the idea is to create a dataset for a siamese network. We have to create a list of tuples (image1, image2, label) where label is 1 if the two images are of the same person and 0 otherwise
#if there is a person with only one image, we discard use it for the training in the false case

#create a dictionary with the name of the person as key and the list of images as value
diz = {}
for folder in folders:
    #check if its actually a folder
    if not os.path.isdir(pathLFW + folder):
        continue
    #read the RGB images
    images = [cv2.imread(pathLFW + folder + '/' + img) for img in os.listdir(pathLFW + folder) if img.endswith('.jpg')]
    diz[folder] = images


train_x = []
train_y = []


print('Number of people: ', len(diz))

firstPair = False



for nome in diz.keys():
    if len(diz[nome]) == 1:
        #print("ciao")
        if not firstPair:   #if it was already been found a person with only one image then we have created a tuple
            img1 = diz[nome][0]
            #resize the image into a 160x160 image

        


            firstPair = True
            
        else:
            img2 = diz[nome][0]
            #resize the image into a 160x160 image
            
            tupla = (img1, img2)
            train_x.append(tupla)
            train_y.append(0)
            firstPair = False
    else:
        #print("ciao2")
        #create a for with step 2
        for i in range(0, len(diz[nome]), 2):
            #check if i + 1 is out of range
            if i + 1 >= len(diz[nome]):
                if not firstPair:   #if it was already been found a person with only one image then we have created a tuple
                    img1 = diz[nome][i]
                    
                    firstPair = True
                    
                else:
                    img2 = diz[nome][i]
                    
                    tupla = (img1, img2)
                    train_x.append(tupla)
                    train_y.append(0)
                    firstPair = False
                continue
            img1 = diz[nome][i]
           

            img2 = diz[nome][i+1]
        
            tupla = (img1, img2)
            train_x.append(tupla)
            train_y.append(1)


print('Number of 0 labels: ', train_y.count(0))
print('Number of 1 labels: ', train_y.count(1))

#train x in a numpy array
train_x = np.array(train_x)
train_y = np.array(train_y)






            

    



Number of people:  99
Number of 0 labels:  40
Number of 1 labels:  43


In [7]:
embedder = FaceNet()

#set verbose to 0 to avoid printing the progress bar in tensorflow

with io.capture_output(stderr=False) as captured:
    train_embeddings_x = []
    for i in range(train_x.shape[0]):
        # Print to stderr
        print(i, file=sys.stderr)
        
        print(train_x[i][0].shape)
        print(train_x[i][1].shape)
        emb1 = embedder.extract(train_x[i][0], threshold=0.95)
        emb2 = embedder.extract(train_x[i][1], threshold=0.95)
        train_embeddings_x.append((emb1, emb2))

    train_embeddings_x = np.array(train_embeddings_x)

print(train_embeddings_x.shape)





0
2023-01-27 13:01:37.941467: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-01-27 13:01:38.956810: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-01-27 13:01:38.957903: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2023-01-27 13:01:38.957914: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at ptxas
2023-01-27 13:01:38.957958: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilatio

(83, 2)


/tmp/ipykernel_1191530/465954047.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_embeddings_x = np.array(train_embeddings_x)


In [8]:
#save train embeddings and train y
np.save('./embeddings/train_embeddings_x.npy', train_embeddings_x)
np.save('./embeddings/train_y.npy', train_y)